In [1]:
from ast import literal_eval
from collections import Counter
from datetime import datetime
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, classification_report

from sherlock.deploy.model import SherlockModel
from sherlock.features.helpers import generate_chars_col
from sklearn import *
import os
import gdown
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import seaborn as sns
import matplotlib.pyplot as plt
#from matplotlib import pyplot as plt
from pyarrow.parquet import ParquetFile
#from googletrans import Translator
import string
import pyarrow.lib
import ast
import re
import string
import csv
import io
import statistics as statistics
import itertools
import math as math
%matplotlib inline
# set seed for reproducibility
np.random.seed(0) 
from collections import OrderedDict
from nltk.stem.snowball import SnowballStemmer
import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from functional import pseq, seq
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from typing import Union, Tuple
import nltk
from tqdm import tqdm
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    BatchNormalization,
    concatenate,
)
from tensorflow.keras.models import Model, model_from_json

from sherlock.deploy import helpers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder



In [2]:
model_id = "sherlock2.2" #retreinar sherlock

In [3]:

X= pd.read_parquet('feature_vectors_final3_novo.parquet')
y=pd.read_parquet('labels_final3_novo.parquet').values.flatten()

In [4]:
# Separa o nosso conjunto de dados train (20%/80%)
X_train, X_, y_train, y_= train_test_split( X, y, train_size=0.80,random_state=0)
print (f"train: {len(X_train)} ({(len(X_train) / len(X)):.2f})\n"
       f"remaining: {len(X_)} ({(len(X_) / len(X)):.2f})")

train: 305 (0.80)
remaining: 77 (0.20)


In [5]:
X_val, X_test, y_val, y_test = train_test_split( X, y, test_size=0.5,random_state=0)

print(f"train: {len(X_train)} ({len(X_train)/len(X):.2f})\n"
      f"val: {len(X_val)} ({len(X_val)/len(X):.2f})\n"
      f"test: {len(X_test)} ({len(X_test)/len(X):.2f})")

train: 305 (0.80)
val: 191 (0.50)
test: 191 (0.50)


In [6]:
y_train = np.array([x.lower() for x in y_train])
y_val = np.array([x.lower() for x in y_val])
y_test = np.array([x.lower() for x in y_test])


In [7]:
y_train = np.array([x.strip() for x in y_train]) 
y_val = np.array([x.strip() for x in y_val]) 
y_test = np.array([x.strip() for x in y_test])

In [8]:
print('Distinct types for columns in the Dataframe (should be all float32):')
print(set(X_train.dtypes))
print(set(X_val.dtypes))
print(set(X_test.dtypes))

Distinct types for columns in the Dataframe (should be all float32):
{dtype('float32')}
{dtype('float32')}
{dtype('float32')}


In [12]:
model_id = "sherlock2.2"
start = datetime.now()
print(f'Started at {start}')

model = SherlockModel()
# Model will be stored with ID `model_id`
model.fit(X_train, y_train, X_val, y_val, model_id=model_id)

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-06-07 00:21:15.109550
Train on 305 samples, validate on 191 samples
Epoch 1/100
305/305 [==============================] - 1s 5ms/sample - loss: 5.4071 - categorical_accuracy: 0.1148 - val_loss: 5.2481 - val_categorical_accuracy: 0.2356
Epoch 2/100
305/305 [==============================] - 1s 3ms/sample - loss: 4.6277 - categorical_accuracy: 0.2918 - val_loss: 4.8653 - val_categorical_accuracy: 0.3351
Epoch 3/100
305/305 [==============================] - 1s 3ms/sample - loss: 4.2670 - categorical_accuracy: 0.3115 - val_loss: 4.5810 - val_categorical_accuracy: 0.3246
Epoch 4/100
305/305 [==============================] - 1s 4ms/sample - loss: 4.0150 - categorical_accuracy: 0.3213 - val_loss: 4.1921 - val_categorical_accuracy: 0.3351
Epoch 5/100
305/305 [==============================] - 1s 3ms/sample - loss: 3.9352 - categorical_accuracy: 0.3082 - val_loss: 3.8640 - val_categorical_accuracy: 0.3455
Epoch 6/100
305/305 [==============================] - 1s 3ms/sample - 

Epoch 49/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.6052 - categorical_accuracy: 0.6230 - val_loss: 0.9049 - val_categorical_accuracy: 0.8063
Epoch 50/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.5793 - categorical_accuracy: 0.6361 - val_loss: 0.8899 - val_categorical_accuracy: 0.8168
Epoch 51/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.5984 - categorical_accuracy: 0.6066 - val_loss: 0.8931 - val_categorical_accuracy: 0.8063
Epoch 52/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.6184 - categorical_accuracy: 0.6131 - val_loss: 0.8987 - val_categorical_accuracy: 0.8063
Epoch 53/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.6162 - categorical_accuracy: 0.6328 - val_loss: 0.8518 - val_categorical_accuracy: 0.8168
Epoch 54/100
305/305 [==============================] - 1s 3ms/sample - loss: 1.4685 - categorical_accuracy: 0.6393 - val_loss: 0.8366 - val_categoric

In [13]:
predicted_labels = model.predict(X_test,model_id)
predicted_labels = np.array([x.lower() for x in predicted_labels])

In [14]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

prediction count 191, type = <class 'numpy.ndarray'>


0.5977836312459577